In [32]:
import os
import math
import pandas as pd

def calculate_h_index(quotation): #h_ndex
    quotation.sort(reverse=True)  # 논문을 내림차순으로 정렬

    h_index = 0
    for i, quotation in enumerate(quotation):
        if quotation >= i + 1:
            h_index = i + 1
        else:
            break

    return h_index

def calculate_i10_index(quotation):  #i10_index
    i10_index = sum(1 for quotation in quotation if quotation >= 10)
    return i10_index

def calculate_g_index(quotation):  #g_index
    quotation.sort(reverse=True)  # 논문을 내림차순으로 정렬

    g_index = 0
    quotation_sum=0
    for i, quotation in enumerate(quotation):
        quotation_sum+=quotation
        if quotation_sum >= (i + 1)**2:
            g_index = i + 1

    return g_index

#Object 타입에서 숫자형 체크용
def is_numeric(value):
    try:
        float(value)  # float으로 변환 가능한 경우 숫자
        return True
    except (ValueError, TypeError):
        return False

def recentness(pYears):
    # dt = datetime.datetime.now()#현재날짜
    dt = 2013
    rct = 0
    lct = 0
    if len(pYears) == 0:
        rct = 0
        lct = 0
    else:
        year_avg = sum(pYears) / len(pYears)
        if year_avg >= int(dt)-2: #int(dt.year)-2:
            lct = 1
        # else:
        #     lct = max(round((1-(int(dt)-3-year_avg)*0.1),2), 0)
        elif int(dt)-15 < year_avg <= int(dt)-3:
            lct = max(round((1-(int(dt)-3-year_avg)*0.1),2), 0)

        rct += len(pYears) #연도 개수

    return rct, lct

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', None)

In [33]:
#authors.all 파일 위치 주소
authors_file_path = r"C:\Users\isjun\바탕 화면\연구실\전문가 검색\gsc_data\gsc_data\authors.all"

try:
    with open(authors_file_path, 'r', encoding='cp1252') as file:
        lines = file.read().splitlines()
        data = [line.split('|') for line in lines]
        columns = ["Valid", "Name", "Affiliation", "Email Domain", "Citation Count", "ID"]
        df_a = pd.DataFrame(data,columns=columns) # authors 의 a
        # df_a = df_a.drop(columns=["C6", "C7"])
        print("변환 완료")
except FileNotFoundError:
    print("파일을 찾을 수 없습니다.")
except UnicodeDecodeError:
    print("파일을 ANSI(Windows-1252) 인코딩으로 열 수 없습니다.")
    
df_a_copy = df_a.copy()  # df_a의 복사본 생성

for index, row in df_a_copy.iterrows():
    if pd.isna(row[5]):  
        if pd.isna(row[4]):
            if pd.isna(row[3]):  # 1의 경우
                row[5] = row[2] 
            else:  # 2,3,4 의 경우
                row[5] = row[3]
                if is_numeric(row[2]): # 4의 경우
                    row[4] = row[2]
                    row[2] = None
                    row[3] = None
                elif '.' in str(row[2]):  # 3의 경우
                    row[3] = row[2]
                    row[2] = None
                    row[4] = None
                else: # 2의 경우
                    row[3] = None
        else:  # 5,6,7 의 경우
            row[5] = row[4] 
            if is_numeric(row[3]): # 6,7 의 경우
                row[4] = row[3]
                if '.' in str(row[2]): # 7 의 경우
                    row[3] = row[2]
                    row[2] = None
                else: # 6의 경우
                    
                    row[3] = None
            else: #5의 경우
                row[4] = None

변환 완료


In [34]:
#gsc_data\DATA 폴더 경로
folder_path = r"C:\Users\isjun\바탕 화면\연구실\전문가 검색\gsc_data\gsc_data\DATA"

max_scoquality = 62888.30906843457
max_rct = 15106

max_lct = []

# df_a_copy의 각 행에서 1번 열의 값을 추출
for index, row in df_a_copy.iterrows():
    name = row[5]  # 1번 열의 값
    
    # 파일 경로 생성
    file_path = os.path.join(folder_path, name + "_.dat")
    
    # 파일이 존재하는지 확인
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            # 파일의 각 줄을 읽어 리스트로 저장
            lines = file.readlines()

        # 각 줄을 나누어서 리스트로 저장
        line_lists = [line.strip().split('|') for line in lines]
        
        #빈 리스트 생성
        citations = [] #인용수
        years = [] #연도
        scoquality = 0 #품질
        for line_list in line_lists:
            if len(line_list) > 0:
                citations.append(int(line_list[0]))
                years.append(int(line_list[1]))
                scoquality += math.log(int(line_list[0]*1)+1)
        
        # print(name, years)
        rct, lct = recentness(years)
        # max_lct.append(rct)
        norm_scoquality = scoquality/max_scoquality
        # norm_recentness = rct/max_rct*0.66*lct
        norm_recentness = ((rct/max_rct)+lct)*0.66
        score = (norm_scoquality + norm_recentness)*50
        # print(rct, scoquality, scoquality/max_scoquality, rct/max_rct, lct, norm_recentness, score)
        #각 index 계산 후 df_a_copy에 새로운 열로 추가
        df_a_copy.at[index,"Paper Count"] = len(citations)
        df_a_copy.at[index,"h-index"] = calculate_h_index(citations)
        df_a_copy.at[index,"i10-index"] = calculate_i10_index(citations)
        df_a_copy.at[index,"g-index"] = calculate_g_index(citations)
        df_a_copy.at[index,"scoquality"] = norm_scoquality
        df_a_copy.at[index,"recentness"] = norm_recentness
        df_a_copy.at[index,"score"] = score
    
    else:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        
df_a_copy['ID'] = 'id_' + df_a_copy['ID']

# print(df_a_copy)
df_a_copy.head(20)
df_a_copy.to_csv('output.csv', index=False)
df_a_copy.to_csv('output_index.csv', index=True)

In [35]:
import csv
import pandas as pd

# CSV 파일 경로
raw_data = 'output.csv'
df = pd.read_csv(raw_data)

#지수별로 내림차순 정렬
H_sorted=df.sort_values(by='h-index', ascending=False)
I10_sorted=df.sort_values(by='i10-index', ascending=False)
G_sorted=df.sort_values(by='g-index', ascending=False)
Citation_sorted=df.sort_values(by='Citation Count', ascending=False)
PaperC_sorted=df.sort_values(by='Paper Count', ascending=False)
scoquality_sorted=df.sort_values(by='scoquality', ascending=False)
recentness_sorted=df.sort_values(by='recentness', ascending=False)
score_sorted=df.sort_values(by='score', ascending=False)

#필요없는 컬럼 삭제
H_sorted=H_sorted.reset_index(drop=True, inplace=False)
I10_sorted=I10_sorted.reset_index(drop=True, inplace=False)
G_sorted=G_sorted.reset_index(drop=True, inplace=False)
Citation_sorted=Citation_sorted.reset_index(drop=True, inplace=False)
PaperC_sorted=PaperC_sorted.reset_index(drop=True, inplace=False)
scoquality_sorted=scoquality_sorted.reset_index(drop=True, inplace=False)
recentness_sorted=recentness_sorted.reset_index(drop=True, inplace=False)
score_sorted=score_sorted.reset_index(drop=True, inplace=False)

#지수 순위 입력
# num=int(input("숫자 입력 : "))
num=50

#해당 순위이내의 결과만 남김
H_sorted = H_sorted.head(num)
I10_sorted = I10_sorted.head(num)
G_sorted = G_sorted.head(num)
Citation_sorted = Citation_sorted.head(num)
PaperC_sorted = PaperC_sorted.head(num)
scoquality_sorted = scoquality_sorted.head(num)
recentness_sorted = recentness_sorted.head(num)
score_sorted = score_sorted.head(num)

#csv 파일 저장
# H_sorted.to_csv('H_sorted.csv')
# I10_sorted.to_csv('I10_sorted.csv')
# G_sorted.to_csv('G_sorted.csv')
# Citation_sorted.to_csv('Citation_sorted.csv')
# PaperC_sorted.to_csv('PaperC_sorted.csv')
# scoquality_sorted.to_csv('scoquality_sorted.csv')
# recentness_sorted.to_csv('recentness_sorted.csv')
# score_sorted.to_csv('score_sorted.csv')

Combine=[H_sorted, I10_sorted, G_sorted]

Combined_df=pd.concat(Combine, ignore_index=True)

name_count=Combined_df['Name'].value_counts()
ID_count=Combined_df['ID'].value_counts()

ID_count_df = pd.DataFrame({'ID': ID_count.index, 'Count': name_count.values})
# ID_count.to_csv('ID_count.csv')
print(Combined_df)

   Valid                      Name                                        Affiliation                                      Email Domain  Citation Count               ID  Paper Count  h-index  i10-index  g-index  scoquality  recentness      score
0      +  Graham Colditz, MD, DrPH  Professor, Dept. of Surgery, Washington Univer...                                 wudosis.wustl.edu        156961.0  id_M5_mEHQAAAAJ       1201.0    213.0      825.0    366.0    0.063272    0.059073   6.117256
1      -           Pierre Bourdieu  Sociology, Centre de Sociologie Europï¿½enne, ...                                               NaN        253192.0  id_d_lp40IAAAAJ        929.0    180.0      583.0    501.0    0.047448    0.040589   4.401835
2      -           Michel Foucault                    Philosophy, Collï¿½ge de France                                               NaN        274933.0  id_AKqYlxMAAAAJ        672.0    163.0      441.0    524.0    0.037598    0.029361   3.347912
3      +        